In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB 

In [ ]:
!pip install PyAudio

  Created wheel for PyAudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52609 sha256=6d7c5b9923230d85a7606f1eb35ada0f6d5297d2f4938a01cc1486f769563e36
  Stored in directory: /root/.cache/pip/wheels/40/2e/4c/b71e7e96c861a46e6213bc6bb482b94dcf293a92c5e736c1ec
Successfully built PyAudio


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
!unzip Ravdess.zip

Archive:  Ravdess.zip
replace Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
pip install soundfile

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/gdrive/My Drive/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
pip install scikit-learn==0.19.1

     |████████████████████████████████| 9.5 MB 4.8 MB/s 
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
    Running setup.py install for scikit-learn ... error
  Rolling back uninstall of scikit-learn
  Moving to /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~cikit_learn-1.0.2.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/
   from /usr/local/lib/python3.7/dist-packages/~cikit_learn.libs
  Moving to /usr/local/lib/python3.7/dist-packages/sklearn/
   from /usr/local/lib/python3.7/dist-packages/~klearn
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-ins

In [ ]:
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.25)
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

(432, 192)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 73.44%


In [ ]:
from sklearn.svm import SVC 
clf = SVC(kernel='linear') 
clf.fit(x_train,y_train)



SVC(kernel='linear')

In [ ]:
y_pred_new=model.predict(x_test)

In [ ]:
print(model.score(x_test,y_test)*100)

73.4375


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
model = RandomForestClassifier(n_estimators=100,max_depth=5)
model.fit(x_train,y_train)



RandomForestClassifier(max_depth=5)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions

array(['fearful', 'calm', 'calm', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'fearful', 'fearful', 'fearful', 'fearful', 'happy',
       'calm', 'happy', 'disgust', 'fearful', 'calm', 'calm', 'calm',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'disgust', 'fearful',
       'calm', 'happy', 'calm', 'disgust', 'calm', 'happy', 'calm',
       'fearful', 'disgust', 'disgust', 'happy', 'calm', 'calm', 'calm',
       'fearful', 'disgust', 'disgust', 'fearful', 'happy', 'calm',
       'calm', 'happy', 'disgust', 'fearful', 'disgust', 'happy', 'happy',
       'fearful', 'happy', 'happy', 'fearful', 'calm', 'calm', 'calm',
       'calm', 'disgust', 'disgust', 'disgust', 'calm', 'calm', 'calm',
       'fearful', 'disgust', 'calm', 'fearful', 'happy', 'happy', 'calm',
       'disgust', 'calm', 'fearful', 'disgust', 'fearful', 'disgust',
       'fearful', 'calm', 'disgust', 'happy', 'fearful', 'disgust',
       'calm', 'fearful', 'disgust', 'fearful', 'happy', 'happy', 'happy',
   

In [ ]:
print(metrics.classification_report(predictions,y_test))

              precision    recall  f1-score   support

        calm       0.92      0.70      0.80        64
     disgust       0.40      0.61      0.48        36
     fearful       0.57      0.62      0.59        42
       happy       0.50      0.42      0.46        50

    accuracy                           0.59       192
   macro avg       0.60      0.59      0.58       192
weighted avg       0.63      0.59      0.60       192



In [ ]:
print(model.score(x_test,y_test)*100)

59.375
